# Part 0: Using matminer to extract features from structures



In [ ]:
# Install and import necessary packages
!pip install matminer
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.6/55.6 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.9/51.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 88.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.3/332.3 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.6/118.6 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.0/809.0 kB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.1/739.1 kB 42.0 MB/s eta 0:00:00
  Created wheel for bibtexparser: filename=bibtexparser-1.4.3-py3-none-any.whl size=4354

In [ ]:
import pandas as pd
from pymatgen.core.structure import Structure
from matminer.featurizers.structure import SiteStatsFingerprint
from matminer.featurizers.structure import DensityFeatures
import numpy as np
from matminer.datasets import load_dataset
from pymatgen.analysis.local_env import CrystalNN

In [ ]:
#--- Load and Inspect Data ---
print("Loading dataset to get a test structure...")
df = load_dataset("elastic_tensor_2015")
print(df.head())

Loading dataset to get a test structure...
Fetching elastic_tensor_2015.json.gz from https://ndownloader.figshare.com/files/13220603 to /usr/local/lib/python3.11/dist-packages/matminer/datasets/elastic_tensor_2015.json.gz


Fetching https://ndownloader.figshare.com/files/13220603 in MB: 1.1182079999999999MB [00:00, 247.21MB/s]                


  material_id    formula  nsites  space_group      volume  \
0    mp-10003    Nb4CoSi      12          124  194.419802   
1    mp-10010  Al(CoSi)2       5          164   61.987320   
2    mp-10015       SiOs       2          221   25.952539   
3    mp-10021         Ga       4           63   76.721433   
4    mp-10025      SiRu2      12           62  160.300999   

                                           structure  elastic_anisotropy  \
0  [[0.94814328 2.07280467 2.5112    ] Nb, [5.273...            0.030688   
1  [[0. 0. 0.] Al, [1.96639263 1.13529553 0.75278...            0.266910   
2   [[1.480346 1.480346 1.480346] Si, [0. 0. 0.] Os]            0.756489   
3  [[0.         1.09045794 0.84078375] Ga, [0.   ...            2.376805   
4  [[1.0094265  4.24771709 2.9955487 ] Si, [3.028...            0.196930   

      G_Reuss       G_VRH     G_Voigt     K_Reuss       K_VRH     K_Voigt  \
0   96.844535   97.141604   97.438674  194.267623  194.268884  194.270146   
1   93.939650   96.252

In [ ]:
# --- Create and Apply the Featurizer ---
featurizer = DensityFeatures(desired_features=['density', 'vpa', 'packing fraction'])
df_struct = featurizer.featurize_dataframe(df, col_id='structure', ignore_errors=True)
print(df_struct.head())

DensityFeatures:   0%|          | 0/1181 [00:00<?, ?it/s]

  material_id    formula  nsites  space_group      volume  \
0    mp-10003    Nb4CoSi      12          124  194.419802   
1    mp-10010  Al(CoSi)2       5          164   61.987320   
2    mp-10015       SiOs       2          221   25.952539   
3    mp-10021         Ga       4           63   76.721433   
4    mp-10025      SiRu2      12           62  160.300999   

                                           structure  elastic_anisotropy  \
0  [[0.94814328 2.07280467 2.5112    ] Nb, [5.273...            0.030688   
1  [[0. 0. 0.] Al, [1.96639263 1.13529553 0.75278...            0.266910   
2   [[1.480346 1.480346 1.480346] Si, [0. 0. 0.] Os]            0.756489   
3  [[0.         1.09045794 0.84078375] Ga, [0.   ...            2.376805   
4  [[1.0094265  4.24771709 2.9955487 ] Si, [3.028...            0.196930   

      G_Reuss       G_VRH     G_Voigt  ...  poisson_ratio  \
0   96.844535   97.141604   97.438674  ...       0.285701   
1   93.939650   96.252006   98.564362  ...       0.268

# Part 1:Converting the structure to features in GNN


In [ ]:
# Step 1: Load the Data
import numpy as np
from matminer.datasets import load_dataset
from pymatgen.analysis.local_env import CrystalNN

print("Loading dataset to get a test structure...")
df = load_dataset("elastic_tensor_2015")

# Let's use the first material in the dataset
structure = df['structure'][0]
print(f"--- Loaded Structure: {structure} ---")
print(f"--- Starting Feature Extraction for: {structure.formula} ---")
print(f"Total atoms in structure: {len(structure)}")

Loading dataset to get a test structure...
--- Loaded Structure: Full Formula (Nb8 Co2 Si2)
Reduced Formula: Nb4CoSi
abc   :   6.221780   6.221780   5.022400
angles:  90.000000  90.000000  90.000000
pbc   :       True       True       True
Sites (12)
  #  SP           a         b     c
---  ----  --------  --------  ----
  0  Nb    0.152391  0.333153  0.5
  1  Nb    0.847609  0.666847  0.5
  2  Nb    0.666847  0.152391  0.5
  3  Nb    0.333153  0.847609  0.5
  4  Nb    0.847609  0.333153  0
  5  Nb    0.666847  0.847609  0
  6  Nb    0.333153  0.152391  0
  7  Nb    0.152391  0.666847  0
  8  Co    0         0         0.75
  9  Co    0         0         0.25
 10  Si    0.5       0.5       0.75
 11  Si    0.5       0.5       0.25 ---
--- Starting Feature Extraction for: Nb8 Co2 Si2 ---
Total atoms in structure: 12


In [ ]:
# Step 2: Extract Node (Atom) Features Step-by-Step
print("\n--- Building Node Features Matrix ---")
node_features = []
# We'll loop through all atoms, but only print the details for the first 3 to show the process.
for i, atom in enumerate(structure):
    # For each atom, create a list of its chemical properties.
    features = [
        atom.specie.Z,     # Atomic Number
        atom.specie.row,   # Row in Periodic Table
        atom.specie.group, # Group in Periodic Table
        atom.specie.X      # Electronegativity
    ]
    node_features.append(features)

    # This 'if' block prints the features for the first 3 atoms as they are created.
    if i < 13:
        print(f"Processing Atom {i} ({atom.specie.symbol}): Created feature vector -> {features}")

# Convert the final list to a NumPy array
node_features = np.array(node_features)


--- Building Node Features Matrix ---
Processing Atom 0 (Nb): Created feature vector -> [41, 5, 5, 1.6]
Processing Atom 1 (Nb): Created feature vector -> [41, 5, 5, 1.6]
Processing Atom 2 (Nb): Created feature vector -> [41, 5, 5, 1.6]
Processing Atom 3 (Nb): Created feature vector -> [41, 5, 5, 1.6]
Processing Atom 4 (Nb): Created feature vector -> [41, 5, 5, 1.6]
Processing Atom 5 (Nb): Created feature vector -> [41, 5, 5, 1.6]
Processing Atom 6 (Nb): Created feature vector -> [41, 5, 5, 1.6]
Processing Atom 7 (Nb): Created feature vector -> [41, 5, 5, 1.6]
Processing Atom 8 (Co): Created feature vector -> [27, 4, 9, 1.88]
Processing Atom 9 (Co): Created feature vector -> [27, 4, 9, 1.88]
Processing Atom 10 (Si): Created feature vector -> [14, 3, 14, 1.9]
Processing Atom 11 (Si): Created feature vector -> [14, 3, 14, 1.9]


In [ ]:
# Step 3: Find Edges and Edge Features Step-by-Step
print("\n--- Finding Bonds (Edges) and Bond Lengths (Edge Features) ---")
#Creates an instance of the CrystalNN (Crystal Nearest Neighbors) class
#tool that has already learned how to identify chemical bonds
#based on the geometry and chemistry of thousands of known materials.
cnn = CrystalNN()
edge_src = []
edge_dst = []
edge_features = []

# We'll loop through all atoms, but only print the neighbor search for the first 2.
#site is the PeriodicSite object representing that specific atom.
for i, site in enumerate(structure):
    # For the current atom 'i', find all its neighbors.
    neighbor_info = cnn.get_nn_info(structure, i)
    # print(f"Neighbors for atom {i}", neighbor_info)

    # This 'if' block just adds a header for the atoms we are detailing.
    if i < 13:
        print(f"\nSearching for neighbors of Atom {i} ({site.specie.symbol})...")

    # Loop through each neighbor that was found.
    for neighbor in neighbor_info:
        j = neighbor['site_index']
        dist = neighbor['site'].nn_distance

        # Store the connection and its length.
        edge_src.append(i)
        edge_dst.append(j)
        edge_features.append([dist])

        # This 'if' block prints each bond as it's found for the first 2 atoms.
        if i < 13:
            print(f"  > Found a bond: Atom {i} -> Atom {j} | Length: {dist:.3f} Å")

# Create the final arrays from the lists.
edge_index = np.array([edge_src, edge_dst])
edge_features = np.array(edge_features)


--- Finding Bonds (Edges) and Bond Lengths (Edge Features) ---

Searching for neighbors of Atom 0 (Nb)...
  > Found a bond: Atom 0 -> Atom 8 | Length: 2.602 Å
  > Found a bond: Atom 0 -> Atom 9 | Length: 2.602 Å
  > Found a bond: Atom 0 -> Atom 11 | Length: 2.708 Å
  > Found a bond: Atom 0 -> Atom 10 | Length: 2.708 Å

Searching for neighbors of Atom 1 (Nb)...
  > Found a bond: Atom 1 -> Atom 8 | Length: 2.602 Å
  > Found a bond: Atom 1 -> Atom 9 | Length: 2.602 Å
  > Found a bond: Atom 1 -> Atom 10 | Length: 2.708 Å
  > Found a bond: Atom 1 -> Atom 11 | Length: 2.708 Å

Searching for neighbors of Atom 2 (Nb)...
  > Found a bond: Atom 2 -> Atom 9 | Length: 2.602 Å
  > Found a bond: Atom 2 -> Atom 8 | Length: 2.602 Å
  > Found a bond: Atom 2 -> Atom 10 | Length: 2.708 Å
  > Found a bond: Atom 2 -> Atom 11 | Length: 2.708 Å

Searching for neighbors of Atom 3 (Nb)...
  > Found a bond: Atom 3 -> Atom 9 | Length: 2.602 Å
  > Found a bond: Atom 3 -> Atom 8 | Length: 2.602 Å
  > Found a bond

In [ ]:
# Step 4: Display the Final Summary
print("\n--- Final Feature Matrices Ready for GNN ---")
print(f"Final Node Features Shape: {node_features.shape} (12 atoms, 4 features each)")
print(f"Final Edge Index Shape: {edge_index.shape} (2 rows for start/end, 72 bonds total)")
print(f"Final Edge Features Shape: {edge_features.shape} (72 bonds, 1 feature each)")

print(node_features.shape)


--- Final Feature Matrices Ready for GNN ---
Final Node Features Shape: (12, 4) (12 atoms, 4 features each)
Final Edge Index Shape: (2, 72) (2 rows for start/end, 72 bonds total)
Final Edge Features Shape: (72, 1) (72 bonds, 1 feature each)
(12, 4)


# Part 2: Training a simple GNN for prediction.

The structure_to_graph function (the bridge from pymatgen to PyTorch Geometric).

The model architecture (SimpleGNN).

The training loop.

In [ ]:
# Step 1: Import necessary packages
from pymatgen.core import Structure
from pymatgen.analysis.local_env import CrystalNN

import numpy as np
import torch
import torch.nn.functional as F
from torch.nn import Linear
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.data import Data, DataLoader
from pymatgen.core import Structure
from pymatgen.analysis.local_env import CrystalNN

In [ ]:
# Step 2: Helper Function to Convert Structure to GNN Graph
# This encapsulates the feature extraction process
def structure_to_graph(structure: Structure, target_y: float):
    """Converts a pymatgen Structure to a PyTorch Geometric Data object for GNNs."""
    # Node features: [atomic_number, row, group, electronegativity]
    node_features = []
    for atom in structure:
        features = [atom.specie.Z, atom.specie.row, atom.specie.group, atom.specie.X]
        node_features.append(features)

    # Edge index and edge features (bond lengths)
    cnn = CrystalNN()
    edge_src, edge_dst, edge_features = [], [], []
    for i, site in enumerate(structure):
        for neighbor in cnn.get_nn_info(structure, i):
            edge_src.append(i)
            edge_dst.append(neighbor['site_index'])
            edge_features.append([neighbor['site'].nn_distance])

    # Convert to PyTorch Tensors
    x = torch.tensor(node_features, dtype=torch.float)
    edge_index = torch.tensor([edge_src, edge_dst], dtype=torch.long)
    edge_attr = torch.tensor(edge_features, dtype=torch.float)
    y = torch.tensor([target_y], dtype=torch.float) # The property to predict

    return Data(x=x, edge_index=edge_index, edge_attr=edge_attr, y=y)


In [ ]:
# Step 3: Create a Small Dummy Dataset
# In a real project, you would have hundreds or thousands of these.
print("Creating a dummy dataset of crystals...")
# Structure 1: NaCl (let's pretend its property value is -2.5)
nacl_structure = Structure(
    [[5.64, 0, 0], [0, 5.64, 0], [0, 0, 5.64]],
    ["Na", "Na", "Na", "Na", "Cl", "Cl", "Cl", "Cl"],
    [[0,0,0], [0,.5,.5], [.5,0,.5], [.5,.5,0], [.5,.5,.5], [.5,0,0], [0,.5,0], [0,0,.5]]
)
# Structure 2: Si (let's pretend its property value is 0.5)
si_structure = Structure(
    [[0, 2.73, 2.73], [2.73, 0, 2.73], [2.73, 2.73, 0]],
    ["Si", "Si"],
    [[0, 0, 0], [0.25, 0.25, 0.25]]
)
# Structure 3: LiF (let's pretend its property value is -4.0)
lif_structure = Structure(
    [[4.02, 0, 0], [0, 4.02, 0], [0, 0, 4.02]],
    ["Li", "Li", "Li", "Li", "F", "F", "F", "F"],
    [[0,0,0], [0,.5,.5], [.5,0,.5], [.5,.5,0], [.5,.5,.5], [.5,0,0], [0,.5,0], [0,0,.5]]
)


# Convert all structures to a list of graph Data objects
graph_dataset = [
    structure_to_graph(nacl_structure, target_y=-2.5),
    structure_to_graph(si_structure, target_y=0.5),
    structure_to_graph(lif_structure, target_y=-4.0)
]
print(f"Dataset created with {len(graph_dataset)} graphs.")
print(graph_dataset)
print("-" * 30)

Creating a dummy dataset of crystals...
Dataset created with 3 graphs.
[Data(x=[8, 4], edge_index=[2, 48], edge_attr=[48, 1], y=[1]), Data(x=[2, 4], edge_index=[2, 8], edge_attr=[8, 1], y=[1]), Data(x=[8, 4], edge_index=[2, 48], edge_attr=[48, 1], y=[1])]
------------------------------


/usr/local/lib/python3.11/dist-packages/pymatgen/analysis/local_env.py:4021: UserWarning: No oxidation states specified on sites! For better results, set the site oxidation states in the structure.
  r1 = _get_radius(structure[n])
/usr/local/lib/python3.11/dist-packages/pymatgen/analysis/local_env.py:4023: UserWarning: No oxidation states specified on sites! For better results, set the site oxidation states in the structure.
  r2 = _get_radius(entry["site"])
/usr/local/lib/python3.11/dist-packages/pymatgen/analysis/local_env.py:3935: UserWarning: CrystalNN: cannot locate an appropriate radius, covalent or atomic radii will be used, this can lead to non-optimal results.
  nn_data = self.get_nn_data(structure, n)


In [ ]:
# Step 4: Define the GNN Model
class SimpleGNN(torch.nn.Module):
    def __init__(self, num_node_features, hidden_channels):
        super(SimpleGNN, self).__init__()
        torch.manual_seed(42)
        # GCN layers learn from the local neighborhood of each atom
        self.conv1 = GCNConv(num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        # A final linear layer for the regression task
        self.linear = Linear(hidden_channels, 1) # Output is 1 number

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        # 1. Message passing (convolution) layers
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index).relu()
        # 2. Pooling layer to get a single vector for the whole graph
        x = global_mean_pool(x, batch)
        # 3. Final prediction
        out = self.linear(x)
        return out


In [ ]:
# Step 5: Prepare for Training
# Create a DataLoader to handle batches of graphs
loader = DataLoader(graph_dataset, batch_size=1, shuffle=True)
# Instantiate the model
model = SimpleGNN(num_node_features=4, hidden_channels=64)
# Define the optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.MSELoss() # Mean Squared Error for regression


/usr/local/lib/python3.11/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [ ]:
# Step 6: The Training Loop
print("--- Starting GNN Training ---")
model.train()
for epoch in range(201): # Train for 200 epochs
    epoch_loss = 0
    for data in loader: # Iterate over each graph in the dataset
        optimizer.zero_grad()
        out = model(data)
        loss = criterion(out, data.y.view(-1, 1))
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    if epoch % 50 == 0:
        print(f'Epoch: {epoch:03d}, Avg Loss: {epoch_loss/len(loader):.4f}')

print("--- Training Finished ---")


--- Starting GNN Training ---
Epoch: 000, Avg Loss: 0.6192
Epoch: 050, Avg Loss: 0.0027
Epoch: 100, Avg Loss: 0.0004
Epoch: 150, Avg Loss: 0.0001
Epoch: 200, Avg Loss: 0.0001
--- Training Finished ---


In [ ]:
# Step 7: Use the Trained Model for Prediction
model.eval()
print("\n--- Model Predictions ---")
for data in graph_dataset:
    prediction = model(data)
    print(f"Structure: {data.y.item():>5.1f} (Actual) -> {prediction.item():>8.3f} (Predicted)")


--- Model Predictions ---
Structure:  -2.5 (Actual) ->   -2.508 (Predicted)
Structure:   0.5 (Actual) ->    0.500 (Predicted)
Structure:  -4.0 (Actual) ->   -4.011 (Predicted)


**While the results look perfect, it's crucial to understand their limitation. This model is heavily overfitting.**

**Memorization vs. Learning**: The dataset only contained 3 examples. The model was trained and evaluated on the exact same 3 examples. Essentially, the model has likely just memorized the answers for these three specific crystals.

**Poor Generalization**: This model tells us very little about how it would perform on a new, unseen crystal structure. If you gave it a fourth crystal, like MgO, its prediction would likely be very inaccurate because it hasn't learned the general underlying physics; it has only learned to recognize the three structures it has already seen.

**Next Steps in a Real Project**
In a real research project, the next steps would be to ensure the model can generalize to new data.

**1. Use a Larger Dataset:** You would need a dataset with hundreds or, ideally, thousands of different crystal structures and their corresponding properties.

**2. Create a Train/Validation/Test Split:** You must divide your large dataset into three distinct sets:

Training Set (~80%): The data the model actually learns from.

Validation Set (~10%): Used to check the model's performance during training on data it hasn't seen, which helps tune parameters and prevent overfitting.

Test Set (~10%): Held back until the very end. You use this set only once to get a final, unbiased measure of how well your model performs on completely new data.

**3. Evaluate on the Test Set:** A model is only considered successful if it performs well on the test set, not the training set.